In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import pandas as pd
import os
import matplotlib.pyplot as plt
import cortex
from os.path import join
from collections import defaultdict
import numpy as np
import joblib
from tqdm import tqdm
import sys
from copy import deepcopy
sys.path.append('../notebooks')
from neuro.config import repo_dir, PROCESSED_DIR
from neuro import viz
from neurosynth import term_dict, term_dict_rev, get_neurosynth_flatmaps
import viz
import neurosynth
from cortex import mni
import os
os.environ["FSLDIR"] = "/home/chansingh/fsl"

Note, this notebook requires first running `03_export_qa_flatmaps.ipynb` into `df_qa_dict.pkl` files for each subject.

### compute correlations with qa flatmaps and plot avgs

In [49]:
# setting = 'shapley_neurosynth'
# setting = 'full_neurosynth'
# setting = 'individual_gpt4'
settings = ['full_neurosynth_wordrate']  # shapley_neurosynth, individual_gpt4
setting = settings[0]
# subjects = ['UTS01', 'UTS02', 'UTS03']
subjects = [f'UTS0{i}' for i in range(1, 9)]


# comparison hyperparams
apply_mask = True
frac_voxels_to_keep = 0.1  # 0.10
frac_voxels_to_keep_list = [frac_voxels_to_keep]


flatmaps_qa_dicts_by_subject = neurosynth.load_flatmaps_qa_dicts_by_subject(
    subjects, settings)

# flatmaps_gt_dict_list_subject_mni = {subject: [convert_to_mni_space(flatmaps_gt_dict_list_subject[subject][qs[i]], subject=subject)
#                                                for i in tqdm(range(len(qs)))]
#                                      for subject in ['UTS01', 'UTS02', 'UTS03']}
flatmaps_gt_dict_mni = neurosynth.get_neurosynth_flatmaps(mni=True)
qs = list(set(flatmaps_qa_dicts_by_subject['UTS01'].keys()) & set(
    flatmaps_gt_dict_mni.keys()))

  0%|          | 0/8 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/home/chansingh/fmri/qa_results/processed/S01/full_neurosynth_wordrate.pkl'

### Compute pvals

In [ ]:
pvals_subject = compute_pvals_for_subject(
    corrs_df, 'UTS01', frac_voxels_to_keep_list)
pvals_subject.style.background_gradient().format(precision=3)

### Re-do analysis in MNI space

In [45]:
def convert_to_mni_space(flatmap_arr, subject='UTS01'):
    flatmap_vol = cortex.Volume(
        data=flatmap_arr.flatten(), subject=subject, xfmname=f'{subject}_auto')
    flatmap_to_mni_cached = cortex.db.get_mnixfm(subject, f'{subject}_auto')
    mni_vol = mni.transform_to_mni(flatmap_vol, flatmap_to_mni_cached)
    mni_arr = mni_vol.get_fdata()  # the actual array, shape=(182,218,182)
    return mni_arr


flatmaps_qa_dict_list_subjects = {subject: [flatmaps_qa_dicts_by_subject[subject][q] for q in qs]
                                  for subject in flatmaps_qa_dicts_by_subject.keys()}
# flatmaps_gt_dict_list_subject = {subject: get_neurosynth_flatmaps(subject)
#  for subject in ['UTS01', 'UTS02', 'UTS03']}
flatmaps_qa_dict_list_subjects_mni = {subject: [convert_to_mni_space(flatmaps_qa_dict_list_subjects[subject][i], subject=subject)
                                                for i in tqdm(range(len(qs)))]
                                      for subject in flatmaps_qa_dict_list_subjects.keys()}

  0%|          | 0/23 [00:00<?, ?it/s]

Determinant is > 0: FLIPPING!


  0%|          | 0/23 [00:00<?, ?it/s]

Determinant is > 0: FLIPPING!


  0%|          | 0/23 [00:00<?, ?it/s]

...

  0%|          | 0/23 [00:00<?, ?it/s]

Determinant is > 0: FLIPPING!


  0%|          | 0/23 [00:00<?, ?it/s]

Determinant is > 0: FLIPPING!


100%|██████████| 23/23 [02:26<00:00,  6.35s/it]


In [42]:
flatmaps_gt_list_mni = [flatmaps_gt_dict_mni[q] for q in qs]

In [46]:
def linearly_downsample_with_interpolation(arr, factor=2):
    from scipy.ndimage import zoom
    return zoom(arr, 1/factor, order=1)


corrs_avg_df = defaultdict(list)
for question_idx in tqdm(range(len(qs))):

    # subjects = ['UTS01', 'UTS02', 'UTS03']
    subjects = [f'UTS0{k}' for k in range(1, 9)]
    flatmap_avg_mni = None
    for subject in subjects:
        # flatmap_qa_arr = flatmaps_qa_dict_list_subjects[subject][question_idx]
        # flatmap_qa_mni = convert_to_mni_space(flatmap_qa_arr, subject=subject)
        flatmap_qa_mni = flatmaps_qa_dict_list_subjects_mni[subject][question_idx]
        if flatmap_avg_mni is None:
            flatmap_avg_mni = flatmap_qa_mni
        else:
            flatmap_avg_mni += flatmap_qa_mni

        # flatmap_avg_mini = flatmap_avg_mni[::2, ::2, ::2]

        # flatmap_gt_arr = flatmaps_gt_dict_list_subject[subject][qs[question_idx]]
        # flatmap_gt_mni = convert_to_mni_space(flatmap_gt_arr, subject=subject)
        # flatmap_gt_mni = flatmaps_gt_dict_list_subject_mni[subject][question_idx]
        flatmap_gt_mni = flatmaps_gt_list_mni[question_idx]

        # print('corr', np.corrcoef(flatmap_qa_arr.flatten(),
        #   flatmap_gt_arr.flatten())[0, 1])
        corrs_avg_df[f'corr_{subject}'].append(
            np.corrcoef(linearly_downsample_with_interpolation(flatmap_qa_mni).flatten(), flatmap_gt_mni.flatten())[0, 1])

    flatmap_avg_mni /= len(subjects)
    flatmap_avg_mni = linearly_downsample_with_interpolation(flatmap_avg_mni)
    corrs_avg_df['questions'].append(qs[question_idx])
    # print('shapes', flatmap_avg_mni.shape, flatmap_gt_mni.shape)
    corr_avg = np.corrcoef(flatmap_avg_mni.flatten(),
                           flatmap_gt_mni.flatten())[0, 1]
    corrs_avg_df['corr_avg'].append(corr_avg)
    # corrs_avg_df['flatmap_qa'].append(flatmap_avg_mni)

  0%|          | 0/23 [00:00<?, ?it/s]

100%|██████████| 23/23 [00:21<00:00,  1.08it/s]


In [47]:
df = pd.DataFrame(corrs_avg_df).sort_values(
    'corr_avg', ascending=False).set_index('questions')
# add avg row to bottom
df.loc['avg'] = df.mean()

In [48]:
df.style.background_gradient(axis=None, cmap="coolwarm_r", vmin=-
                             df.abs().max().max(), vmax=df.abs().max().max()).format(precision=3)

,corr_UTS01,corr_UTS02,corr_UTS03,corr_UTS04,corr_UTS05,corr_UTS06,corr_UTS07,corr_UTS08,corr_avg
questions,,,,,,,,,
Does the input mention anything related to navigation?,0.082,0.096,0.090,-0.037,0.075,0.026,0.061,0.076,0.119
Does the input mention anything related to gender?,0.053,0.004,0.000,0.096,0.074,0.003,0.050,0.052,0.072
Does the input mention anything related to calculation?,-0.002,0.045,0.006,0.050,0.041,0.008,0.029,0.001,0.059
Does the input contain a sense of ambiguity?,0.056,0.023,0.015,-0.007,0.034,0.010,0.029,-0.006,0.039
Does the input mention anything related to knowledge?,0.025,0.020,0.028,0.015,-0.015,0.016,-0.000,0.041,0.035
Does the input mention or describe a smell?,0.018,-0.005,0.039,0.042,-0.011,0.022,0.010,-0.025,0.026
Does the input mention anything related to arithmetic?,-0.010,0.035,0.009,0.008,0.008,-0.005,-0.008,0.021,0.018
Does the input mention anything related to diseases?,0.005,0.013,-0.001,0.019,0.021,-0.016,0.000,0.009,0.014
Does the input mention anything related to children?,-0.020,0.030,-0.029,-0.030,0.024,0.035,-0.004,0.029,0.014


### View flatmaps in 1 plot

In [ ]:
corrs_df = pd.read_pickle(join(repo_dir, 'qa_results',
                               'neurosynth', setting + '_corrs_df.pkl'))

In [ ]:
setting = 'shapley_neurosynth'
for subject in ['UTS01', 'UTS02', 'UTS03']:
    img_dir1 = join(repo_dir, 'qa_results', 'neurosynth',
                    subject, 'neurosynth')
    img_dir2 = join(repo_dir, 'qa_results', 'neurosynth',
                    subject, setting)

    # read images and combine them with their filenames on a single plot
    # fnames = os.listdir(img_dir1)
    # fnames = [f for f in fnames if f.endswith('.png')]
    # only keep the ones that are in both directories
    # fnames = [f for f in fnames if f in os.listdir(img_dir2)]

    corrs = corrs_df[corrs_df['subject'] == subject]
    # corrs = corrs.sort_values(f'corrs_{frac_voxels_to_keep}', ascending=False)
    fnames = [v + '.png' for v in corrs['questions'].values]

    n = len(fnames)
    C = 4
    R = int(np.ceil(n / C))

    fig, axs = plt.subplots(R, C, figsize=(C * 3.2, R * 1))
    axs = axs.flatten()
    for i in range(len(axs)):
        axs[i].axis('off')
    for i, fname in enumerate(fnames):
        img1 = plt.imread(join(img_dir1, fname))
        img2 = plt.imread(join(img_dir2, fname))
        axs[i].imshow(np.concatenate([img1, img2], axis=1))
        axs[i].set_title(
            f'{term_dict_rev[fname[:-4]]} ({corrs["corrs"].values[i]:0.3f})', fontsize=8)

    # add text in bottom right of figure
    fig.text(0.99, 0.01, f'{subject}\nNeurosynth on left, QA on right',
             ha='right', va='bottom', fontsize=8)
    plt.tight_layout()
    plt.savefig(join(repo_dir, 'qa_results', 'neurosynth',
                subject, f'flatmaps_{setting}_{subject}.png'), dpi=300)
    plt.close()